In [ ]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()

Pkg.add("MeshCat")
Pkg.add("GeometryBasics")
Pkg.add("CoordinateTransformations")
Pkg.add("DifferentiableCollisions")
Pkg.add("StaticArrays")
Pkg.add("MeshCatMechanisms")
Pkg.add("RigidBodyDynamics")
Pkg.add("LinearAlgebra")
Pkg.add("Rotations")
using Rotations
using LinearAlgebra
using RigidBodyDynamics
using MeshCat
using GeometryBasics
using CoordinateTransformations
using StaticArrays
using MeshCatMechanisms

In [ ]:
function animate_tiltrotor(X, dt)
    vis = Visualizer()
    urdf = joinpath(@__DIR__,"urdf/tiltrotor.urdf")
    robot = parse_urdf(urdf)
    remove_fixed_tree_joints!(robot)

    mvis = MechanismVisualizer(robot, URDFVisuals(urdf), vis)
    settransform!(vis, Translation(0., 0, 3))
    set_configuration!(mvis, [1.5, -1.5])
    anim = Animation(floor(Int,1/dt))
    for k = 1:length(X)
        atframe(anim, k) do
            settransform!(vis, Translation(0., 0, X[k]+1.5) ∘ LinearMap(RotZ(X[k])))  
            set_configuration!(mvis, [X[k], X[k]])
        end
    end
    setanimation!(vis, anim)
    return render(vis)
end
display(animate_tiltrotor(1.5:-0.1:0.7, 0.1))

## Dynamics

Type out dynamics here

In [ ]:
function tiltrotor_dynamics(model::NamedTuple,x,u)
    # Put dynamics here
end

## State Vector

X = [r, v, q, ω, λ_l, λ_r]

- r -> position in world frame
- v -> velocity in world frame
- q -> quaternion rotation
- ω -> angular velocity
- λ_l -> tilt of left rotor
- λ_r -> tilt of right rotor

## Control Vector

U = [ω_l, ω_r, λ_l_dot, λ_r_dot]

- ω_l -> rpm of left rotor
- ω_r -> rpm of right rotor 
- λ_l_dot -> rate of change of left rotor tilt
- λ_r_dot -> rate of change of right rotor tilt

## Integration

In [ ]:
function discrete_dynamics(params::NamedTuple, x::Vector, u, k)
    # discrete dynamics
    # x - state 
    # u - control 
    # k - index of trajectory 
    # dt comes from params.model.dt 
    return rk4(params.model, tiltrotor_dynamics, x, u, params.model.dt)
end

## Cost

DIRCOL or iLQR?

## Trajectory Optimization

In [ ]:
# dynamics parameters (taken from HW3)
model = (mass=0.5,
        J=Diagonal([0.0023, 0.0023, 0.004]),
        gravity=[0,0,-9.81],
        L=0.1750,
        kf=1.0,
        km=0.0245,dt = dt)

## Tracking with TVLQR

## Visualize